In [8]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

def create_frozen_map(lat):
    """ Creates a boolean map of (freezes) all bonds (nn interactions) in the lattice
    
        Input:
            lat: dtype numpy array
                [N] array containing only 1 and -1 values representing the spin configuration
            
        Returns:
            f_map: dtype numpy array
                [N] boolean array containing True if bond is present or False if not. For example spin lattice [1, -1, 1, 1]
                corresponds to f_map [True, True, False, False]
    """
    lat = np.array(lat)
    a = lat[1:]-lat[0:len(lat)-1]; b = lat[0]-lat[len(lat)-1]
    c = np.append(a,b)
    f_map = (c == 0)
    return(f_map)

def backtrack(x,vis,fm,cl):
    """ Part of the cluster decomposition. It recursively checks whether neighbouring spin should be added to the cluster
    """
    if not vis[x]:
        vis[x] = True
        cl.append(x)
        if fm[x]:
            vis, cl = backtrack((x+1)%len(fm),vis,fm,cl)
        if fm[(x-1)%len(fm)]:
            vis, cl = backtrack((x-1)%len(fm),vis,fm,cl)
    return(vis,cl)

def cluster_decomp(lat):
    """ Decomposes the spin lattice into clusters, i.e. creates list of cluster arrays. A cluster array contains the indices of 
        the spins in a cluster
        
        Input:
            lat: dtype numpy array
                [N] array containing only 1 and -1 values representing the spin configuration
                
        Returns:
            cls: dtype list
                list of cluster arrays. Each cluster array contains the indices of the spins in that cluster
    """
    visited = np.full((len(lat)), False)
    frozen_map = create_frozen_map(lat)
    #zeros = np.where(np.array(lat)==0)[0]
    #tried = dict.fromkeys(zeros,False)
    cls = []
    for i in range(len(lat)):
        clu = []
        visited, clu = backtrack(i,visited,frozen_map,clu)
        if len(clu) > 0:
            cls.append(clu)
    return(cls)

def create_boundary_graph(lat):
    """ Create boundary graph (collection of nodes) corresponding to the spin lattice
    
        Input: 
            lat: dtype numpy array
                [N] array containing only 1 and -1 values representing the spin configuration
            
        Returns: 
            bdGr: dtype networkx graph
                boundary graph 
    """
    n_nodes = len(lat)
    bdGr = nx.Graph()
    for i in range(N):
        bdGr.add_node(i)
    return(bdGr)

def construct_graph(lat,bdGr):
    """ Construct graph from spin lattice according to the AoSD procedure
        
        Input: 
            lat: dtype numpy array
                [N] array containing only 1 and -1 values representing the spin configuration
            
            bdGr: dtype networkx graph
                graph containing the boundary nodes (i.e. as constructed by the function 'create_boundary_graph')
            
        Returns: 
            Gr: dtype networkx graph
                AoSD graph 
    """
    lats = []; lats.append(lat)
    shell = 0; n_nodes_shell = len(np.array(lats[shell])); add = 0
    Gr = bdGr.copy()
    CGP_step1 = True
    while(n_nodes_shell>1):
        new_lat = []
        if CGP_step1:
            # perform step 1 of the coarse grain procedure (create domain branches)
            clusters = cluster_decomp(lats[shell])
            # go through clusters
            for i in range(len(clusters)):
                value = 0
                n_Gr = nx.number_of_nodes(Gr); Gr.add_node(n_Gr)
                for j in clusters[i]:
                    value += lats[shell][j]
                    Gr.add_edge(n_Gr,j+add)
                new_lat.append(value)
            CGP_step1 = False
        else:
            # peform step 2 of the coarse grain procedure (average over domains)
            for k in np.arange(0,n_nodes_shell,2):
                value = 0
                n_Gr = nx.number_of_nodes(Gr); Gr.add_node(n_Gr)
                #print('n_Gr', n_Gr)
                #print('k',k+add)
                value = lats[shell][k]+lats[shell][k+1]
                if value == 0:
                    value = np.random.choice([-1,1])
                else:
                    value = value/abs(value)
                Gr.add_edge(n_Gr,k+add); Gr.add_edge(n_Gr,k+1+add)
                new_lat.append(value)
            CGP_step1 = True
        lats.append(new_lat)
        shell += 1
        add += n_nodes_shell; n_nodes_shell = len(np.array(lats[shell]))
    return(Gr)

N = 50 # Global parameter denoting the number of spins (boundary nodes)
lattice = np.ones(N)
bdG = create_boundary_graph(lattice)
G = construct_graph(lattice, bdG)
nx.draw(G)